In [1]:
from datasets import load_from_disk

dataset = load_from_disk("/fs/nexus-projects/HuangWM/datasets/mscoco_5k")
images1 = [dataset[i]["image"] for i in range(0, 5)]
images2 = [dataset[i]["image"] for i in range(5, 10)]

In [ ]:
from metrics import *

print(
    compute_mse_repeated(images1, images2),
    compute_psnr_repeated(images1, images2),
    compute_ssim_repeated(images1, images2),
    compute_lpips_repeated(images1, images2, mode="vgg"),
    compute_watson_repeated(images1, images2, mode="vgg"),
)

In [2]:
from metrics import *

image = dataset[0]["image"]
prompt = dataset[0]["prompt"]

# ppl_model, ppl_tokenizer = load_perplexity_model_and_tokenizer()
# clip_tokenizer = load_open_clip_tokenizer()
(
    clip_model,
    clip_preprocess,
    clip_tokenizer,
) = load_open_clip_model_preprocess_and_tokenizer()


print(
    # compute_prompt_perplexity(prompt, ppl_model, ppl_tokenizer, stride=512),
    # compute_open_clip_num_tokens(prompt, clip_tokenizer),
    compute_clip_score(image, prompt, clip_model, clip_preprocess, clip_tokenizer),
)

7.520791053771973 14 0.15173929929733276
